In [496]:
import jieba
import codecs
from gensim import corpora,models,similarities
import pandas as pd
import numpy as np

In [497]:
#对句子分词、去停用词
def tokenization(str_text):
    res = []
    words = jieba.cut_for_search(str_text)
    #words = jieba.cut(str_text)
    #print(' '.join(words))
    for word in words:
        #去掉不想要的特殊字符
        if word == '(' or word == ')':
            continue
        else:
            res.append(word)
    return res

In [498]:
def similarity_searchStr_forUniversity(search_str,get_search_list):
    str_list = get_search_list
    corpus = []
    for each in str_list:  
        corpus.append(tokenization(each['title']))
    #建立词袋模型，生成字典和向量语料
    print(corpus)
    dictionary = corpora.Dictionary(corpus)
    doc_vectors = [dictionary.doc2bow(text) for text in corpus]
    print('doc_vectors:',doc_vectors)
    #建立tf-idf模型
    tfidf = models.TfidfModel(doc_vectors)
    tfidf_vectors = tfidf[doc_vectors]
    #len(tfidf_vectors) = 文本数  len(tfidf_vectors[0] == 第一个文本中的词数量)
    #print(len(tfidf_vectors))
    
    #构建LSI模型，设置主题数（类数）
    lsi = models.LsiModel(tfidf_vectors,id2word = dictionary,num_topics=2)
    lsi_vector = lsi[tfidf_vectors]
    #LSI向量空间中，所有文本的向量都是二维的
    query = tokenization(search_str)
    query_bow = dictionary.doc2bow(query)
    query_lsi = lsi[query_bow]
    print(query)
    index = similarities.MatrixSimilarity(lsi_vector)
    sims = index[query_lsi]
    print(list(enumerate(sims)))
    
    max_index = list(sims).index(max(sims))
    res_dict = {}
    res_dict.update(search_str = search_str)
    if max(sims) == 0:
        res_dict.update(url = None)
    else:
        res_dict.update(url = get_search_list[max_index]['url'])
    return res_dict

In [499]:
def similarity_searchStr_forCompany(search_str,get_search_list):
    str_list = get_search_list
    corpus = []
    for each in str_list:    
        corpus.append(tokenization(each['title']+each['address']))
    #建立词袋模型，生成字典和向量语料
    print(corpus)
    dictionary = corpora.Dictionary(corpus)
    doc_vectors = [dictionary.doc2bow(text) for text in corpus]
    print('doc_vectors:',doc_vectors)
    #建立tf-idf模型
    tfidf = models.TfidfModel(doc_vectors)
    tfidf_vectors = tfidf[doc_vectors]
    #len(tfidf_vectors) = 文本数  len(tfidf_vectors[0] == 第一个文本中的词数量)
    #print(len(tfidf_vectors))
    
    query = tokenization(search_str)
    print(query)
    query_bow = dictionary.doc2bow(query)
    print(query_bow)
    index = similarities.MatrixSimilarity(tfidf_vectors)
    sims = index[query_bow]
    print(list(enumerate(sims)))
       
    max_index = list(sims).index(max(sims))
   
    res_dict = {}
    res_dict.update(search_str = search_str)
    if max(sims) == 0:
        res_dict.update(url = None)
    else:
        res_dict.update(url = get_search_list[max_index]['url'])
    return res_dict

In [500]:
def toTheMostSimiStr(search,get_search_list):
    search_str = search['title_search']
    query = tokenization(search_str)   
    if '大学' in query or '学校' in query or '学院' in query:
        res_titile = similarity_searchStr_forUniversity(search_str,get_search_list)
    else:
        res_titile = similarity_searchStr_forCompany(search_str,get_search_list)
    return res_titile

In [501]:
get_search_list = [{'title': '中国海关管理干部学院', 'url': 'ss','sdress':''}, {'title': '管理干部学院', 'url': 'dddd'},{'title': '中国海关管理干部学院副院长', 'url': 's'}]

In [502]:
search_str = {'title_search':'海关管理干部学院'}

In [503]:
res = toTheMostSimiStr(search_str,get_search_list)

[['中国', '海关', '中国海', '中国海关', '管理', '干部', '学院'], ['管理', '干部', '学院'], ['中国', '海关', '中国海', '中国海关', '管理', '干部', '学院', '副', '院长']]
doc_vectors: [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(3, 1), (4, 1), (6, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]
['海关', '管理', '干部', '学院']
[(0, 1.0), (1, 0.0), (2, 0.46270883)]


In [479]:
get_search_list1 = [{'title': '中国第一汽车集团哈尔滨轻型车场', 'url': 'ss'}, {'title': '中国成人院校', 'url': 'dddd'},{'title': '中国海关管理干部学院副院长', 'url': 'ddds'}]

In [480]:
search_str1 = '哈尔滨轻型车厂职工大学'

In [483]:
res1 = toTheMostSimiStr(search_str1,get_search_list1)

[['中国', '第一', '汽车', '集团', '汽车集团', '哈尔', '哈尔滨', '轻型', '轻型车', '场'], ['中国', '成人', '院校'], ['中国', '海关', '中国海', '中国海关', '管理', '干部', '学院', '副', '院长']]
doc_vectors: [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)], [(0, 1), (10, 1), (11, 1)], [(0, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]]
['哈尔', '哈尔滨', '轻型', '轻型车', '厂', '职工', '工大', '大学', '职工大学']
[(0, 1.0), (1, 0.0), (2, 0.0)]


In [484]:
search_str3 = '搜易贷'
get_search_list3 = [{'title':'北京信息技术','url':'ssss','address': '天津市滨海新区滨海欣嘉园商业广场五号楼1-18门C53'},{'title': '青岛搜易贷经济信息咨询有限公司', 'url': 'dddd','address': '青岛市滨海新区滨海欣嘉园商业广场五号楼1-18门C53'},{'title': '搜易贷(北京)金融信息服务有限公司', 'url': 'ddds','address': '北京市滨海新区滨海欣嘉园商业广场五号楼1-18门C53'}]

In [486]:
res1 = toTheMostSimiStr(search_str3,get_search_list3)

[['北京', '信息', '技术', '信息技术', '天津', '津市', '天津市', '滨海', '新区', '滨海新区', '滨海', '欣嘉园', '商业', '广场', '商业广场', '五号', '楼', '1', '-', '18', '门', 'C53'], ['青岛', '搜易贷', '经济', '信息', '咨询', '有限', '公司', '有限公司', '青岛', '青岛市', '滨海', '新区', '滨海新区', '滨海', '欣嘉园', '商业', '广场', '商业广场', '五号', '楼', '1', '-', '18', '门', 'C53'], ['搜易贷', '北京', '金融', '信息', '金融信息', '服务', '有限', '公司', '有限公司', '北京', '京市', '北京市', '滨海', '新区', '滨海新区', '滨海', '欣嘉园', '商业', '广场', '商业广场', '五号', '楼', '1', '-', '18', '门', 'C53']]
doc_vectors: [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (8, 1), (9, 1), (12, 1), (14, 1), (15, 1), (16, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (7, 2), (8, 1), (9, 1), (12, 1), (14, 1), (15, 1), (16, 1), (18, 2), (19, 1), (20, 1)

In [487]:
res_titile3

{'search_str': '搜易贷', 'url': 'ddds'}